# PAES 2026 - Base de Datos

Creación de base de datos DuckDB con datos de la PAES (Prueba de Acceso a la Educación Superior) de Chile.

In [ ]:
import duckdb
import pandas as pd
from pathlib import Path

# Rutas
DATA_DIR = Path("../data")
DB_PATH = "../paes.duckdb"

# Crear/conectar a la base de datos
con = duckdb.connect(DB_PATH)
print(f"DuckDB version: {duckdb.__version__}")

## 1. Tabla de Establecimientos

In [ ]:
# Crear tabla de establecimientos desde JSON
con.execute(f"""
    CREATE OR REPLACE TABLE establecimientos (
        rbd INTEGER PRIMARY KEY,           -- Rol Base de Datos (ID único del establecimiento)
        nombre VARCHAR,                    -- Nombre del establecimiento educacional
        cod_region INTEGER,                -- Código de región
        cod_comuna INTEGER,                -- Código de comuna
        cod_pais INTEGER                   -- Código de país (56=Chile)
    );
    
    INSERT INTO establecimientos
    SELECT 
        e.rbd,
        e.nom_rbd,
        e.cod_region,
        e.cod_comuna,
        e.cod_pais
    FROM (
        SELECT unnest(establecimientos) as e
        FROM read_json_auto('{DATA_DIR}/new_schools_list.json')
    );
""")

# Agregar comentarios a la tabla
con.execute("COMMENT ON TABLE establecimientos IS 'Establecimientos educacionales de Chile'")

print(f"Establecimientos cargados: {con.execute('SELECT COUNT(*) FROM establecimientos').fetchone()[0]}")

## 2. Tabla de Comunas y Regiones

In [ ]:
# Cargar comunas desde Excel
df_comunas = pd.read_excel(DATA_DIR / 'Libro_CódigosADM2026_ArchivoC.xlsx', sheet_name='Anexo - ComunasRegiones')

# Registrar DataFrame en DuckDB y crear tabla
con.execute("""
    CREATE OR REPLACE TABLE comunas (
        cod_region INTEGER,                -- Código de región (1-16)
        region VARCHAR,                    -- Nombre de la región
        cod_provincia INTEGER,             -- Código de provincia
        provincia VARCHAR,                 -- Nombre de la provincia
        cod_comuna INTEGER PRIMARY KEY,    -- Código de comuna (único)
        comuna VARCHAR                     -- Nombre de la comuna
    )
""")

con.execute("""
    INSERT INTO comunas
    SELECT 
        "COD REG." as cod_region,
        "REGION NOMBRE" as region,
        "COD.PRV" as cod_provincia,
        "PRV NOMBRE" as provincia,
        "COD.COMUNA" as cod_comuna,
        "COM NOMBRE" as comuna
    FROM df_comunas
""")

con.execute("COMMENT ON TABLE comunas IS 'División político-administrativa de Chile: regiones, provincias y comunas'")

print(f"Comunas cargadas: {con.execute('SELECT COUNT(*) FROM comunas').fetchone()[0]}")
con.execute("SELECT * FROM comunas LIMIT 5").df()

## 3. Tabla de Códigos de Enseñanza

In [ ]:
# Cargar códigos de enseñanza desde Excel
df_ensenanza = pd.read_excel(DATA_DIR / 'Libro_CódigosADM2026_ArchivoC.xlsx', sheet_name='Anexo - COD_ENS')

con.execute("""
    CREATE OR REPLACE TABLE cod_ensenanza (
        codigo INTEGER PRIMARY KEY,        -- Código MINEDUC de tipo de enseñanza
        descripcion VARCHAR                -- Descripción del tipo de enseñanza
    )
""")

con.execute("""
    INSERT INTO cod_ensenanza
    SELECT 
        "Código" as codigo,
        "Descripción" as descripcion
    FROM df_ensenanza
""")

con.execute("COMMENT ON TABLE cod_ensenanza IS 'Códigos de enseñanza MINEDUC (HC=Humanista-Científico, TP=Técnico-Profesional)'")

print(f"Códigos de enseñanza cargados: {con.execute('SELECT COUNT(*) FROM cod_ensenanza').fetchone()[0]}")
con.execute("SELECT * FROM cod_ensenanza").df()

## 4. Tablas de Referencia (Dependencia, Rama, Situación Egreso)

In [ ]:
# Crear tablas de referencia para códigos categóricos

# Dependencia administrativa
con.execute("""
    CREATE OR REPLACE TABLE ref_dependencia (
        codigo INTEGER PRIMARY KEY,
        descripcion VARCHAR
    );
    INSERT INTO ref_dependencia VALUES
        (1, 'Particular Pagado'),
        (2, 'Particular Subvencionado'),
        (3, 'Municipal'),
        (4, 'Servicio Local de Educación');
    COMMENT ON TABLE ref_dependencia IS 'Dependencia administrativa del establecimiento según norma universidades';
""")

# Rama educacional
con.execute("""
    CREATE OR REPLACE TABLE ref_rama (
        codigo VARCHAR PRIMARY KEY,
        descripcion VARCHAR
    );
    INSERT INTO ref_rama VALUES
        ('H1', 'Humanista Científico Diurno'),
        ('H2', 'Humanista Científico Nocturno'),
        ('H3', 'Humanista Científico - Validación de estudios'),
        ('H4', 'Humanista Científico - Reconocimiento de estudios'),
        ('T1', 'Técnico Profesional Comercial'),
        ('T2', 'Técnico Profesional Industrial'),
        ('T3', 'Técnico Profesional Servicios y Técnica'),
        ('T4', 'Técnico Profesional Agrícola'),
        ('T5', 'Técnico Profesional Marítima');
    COMMENT ON TABLE ref_rama IS 'Rama educacional del establecimiento de egreso';
""")

# Situación de egreso
con.execute("""
    CREATE OR REPLACE TABLE ref_situacion_egreso (
        codigo INTEGER PRIMARY KEY,
        descripcion VARCHAR
    );
    INSERT INTO ref_situacion_egreso VALUES
        (1, 'Promoción Año: Nacional - alumno regular'),
        (2, 'Promoción Año: Nacional - validación estudios'),
        (3, 'Promoción Año: Extranjero - con curso en Chile'),
        (4, 'Promoción Año: Extranjero - no acredita notas'),
        (5, 'Promoción Anterior: Nacional - alumno regular'),
        (6, 'Promoción Anterior: Nacional - validación estudios'),
        (7, 'Promoción Anterior: Extranjero - con curso en Chile'),
        (8, 'Promoción Anterior: Extranjero - no acredita notas');
    COMMENT ON TABLE ref_situacion_egreso IS 'Tipificación de estudios cursados por el postulante';
""")

# Módulos de ciencias
con.execute("""
    CREATE OR REPLACE TABLE ref_modulo_ciencias (
        codigo VARCHAR PRIMARY KEY,
        descripcion VARCHAR
    );
    INSERT INTO ref_modulo_ciencias VALUES
        ('BIO', 'Biología'),
        ('FIS', 'Física'),
        ('QUI', 'Química'),
        ('TEC', 'Técnico Profesional');
    COMMENT ON TABLE ref_modulo_ciencias IS 'Módulo rendido en prueba de Ciencias';
""")

print("Tablas de referencia creadas: ref_dependencia, ref_rama, ref_situacion_egreso, ref_modulo_ciencias")

## 5. Tabla Principal: Resultados PAES

In [ ]:
# Crear tabla de resultados PAES desde CSV
con.execute("""
    CREATE OR REPLACE TABLE resultados_paes (
        -- Identificación
        id VARCHAR PRIMARY KEY,                    -- Identificador único del postulante
        rbd INTEGER,                               -- RBD del establecimiento de egreso
        cod_ensenanza INTEGER,                     -- Código de enseñanza MINEDUC
        dependencia INTEGER,                       -- Dependencia: 1=Part.Pagado, 2=Part.Subv, 3=Municipal, 4=SLE
        rama VARCHAR,                              -- Rama educacional: H1-H4 (humanista), T1-T5 (técnico)
        situacion_egreso INTEGER,                  -- Situación de egreso (1-8)
        cod_region INTEGER,                        -- Código de región del establecimiento
        cod_comuna INTEGER,                        -- Código de comuna del establecimiento
        
        -- Notas de Enseñanza Media
        promedio_notas DECIMAL(4,2),               -- Promedio de notas de enseñanza media
        percentil_notas INTEGER,                   -- Percentil superior de notas (0-100)
        puntaje_nem INTEGER,                       -- Puntaje NEM estandarizado (100-1000)
        puntaje_ranking INTEGER,                   -- Puntaje Ranking estandarizado (100-1000)
        
        -- PAES Regular Actual (Proceso 2026)
        lectora_reg INTEGER,                       -- Competencia Lectora Regular
        mate1_reg INTEGER,                         -- Matemática 1 Regular
        mate2_reg INTEGER,                         -- Matemática 2 Regular
        historia_reg INTEGER,                      -- Historia y Cs. Sociales Regular
        ciencias_reg INTEGER,                      -- Ciencias Regular
        modulo_reg VARCHAR,                        -- Módulo Ciencias Regular (BIO/FIS/QUI/TEC)
        
        -- PAES Invierno Actual (Proceso 2026)
        lectora_inv INTEGER,                       -- Competencia Lectora Invierno
        mate1_inv INTEGER,                         -- Matemática 1 Invierno
        mate2_inv INTEGER,                         -- Matemática 2 Invierno
        historia_inv INTEGER,                      -- Historia y Cs. Sociales Invierno
        ciencias_inv INTEGER,                      -- Ciencias Invierno
        modulo_inv VARCHAR,                        -- Módulo Ciencias Invierno (BIO/FIS/QUI/TEC)
        
        -- PAES Regular Anterior (Proceso 2025)
        lectora_reg_ant INTEGER,                   -- Competencia Lectora Regular Anterior
        mate1_reg_ant INTEGER,                     -- Matemática 1 Regular Anterior
        mate2_reg_ant INTEGER,                     -- Matemática 2 Regular Anterior
        historia_reg_ant INTEGER,                  -- Historia y Cs. Sociales Regular Anterior
        ciencias_reg_ant INTEGER,                  -- Ciencias Regular Anterior
        modulo_reg_ant VARCHAR,                    -- Módulo Ciencias Regular Anterior
        
        -- PAES Invierno Anterior (Proceso 2025)
        lectora_inv_ant INTEGER,                   -- Competencia Lectora Invierno Anterior
        mate1_inv_ant INTEGER,                     -- Matemática 1 Invierno Anterior
        mate2_inv_ant INTEGER,                     -- Matemática 2 Invierno Anterior
        historia_inv_ant INTEGER,                  -- Historia y Cs. Sociales Invierno Anterior
        ciencias_inv_ant INTEGER,                  -- Ciencias Invierno Anterior
        modulo_inv_ant VARCHAR,                    -- Módulo Ciencias Invierno Anterior
        
        -- Indicadores de participación
        rindio_anterior BOOLEAN,                   -- Rindió al menos una prueba proceso 2025
        rindio_actual INTEGER                      -- Proceso 2026: 1=Regular, 2=Invierno, 3=Ambas
    );
""")

print("Tabla resultados_paes creada. Cargando datos...")

In [ ]:
# Cargar datos desde CSV
con.execute(f"""
    INSERT INTO resultados_paes
    SELECT 
        -- Identificación
        ID_aux as id,
        CAST(RBD AS INTEGER) as rbd,
        CAST(COD_ENS AS INTEGER) as cod_ensenanza,
        CAST(GRUPO_DEPENDENCIA AS INTEGER) as dependencia,
        RAMA_EDUCACIONAL as rama,
        SITUACION_EGRESO as situacion_egreso,
        CAST(CODIGO_REGION AS INTEGER) as cod_region,
        CAST(CODIGO_COMUNA AS INTEGER) as cod_comuna,
        
        -- Notas (decimal_separator=',' ya maneja el formato)
        CAST(PROMEDIO_NOTAS AS DECIMAL(4,2)) as promedio_notas,
        PORC_SUP_NOTAS as percentil_notas,
        PTJE_NEM as puntaje_nem,
        PTJE_RANKING as puntaje_ranking,
        
        -- PAES Regular Actual
        NULLIF(CAST(CLEC_REG_ACTUAL AS INTEGER), 0),
        NULLIF(CAST(MATE1_REG_ACTUAL AS INTEGER), 0),
        NULLIF(CAST(MATE2_REG_ACTUAL AS INTEGER), 0),
        NULLIF(CAST(HCSOC_REG_ACTUAL AS INTEGER), 0),
        NULLIF(CAST(CIEN_REG_ACTUAL AS INTEGER), 0),
        NULLIF(MODULO_REG_ACTUAL, ''),
        
        -- PAES Invierno Actual
        NULLIF(CAST(CLEC_INV_ACTUAL AS INTEGER), 0),
        NULLIF(CAST(MATE1_INV_ACTUAL AS INTEGER), 0),
        NULLIF(CAST(MATE2_INV_ACTUAL AS INTEGER), 0),
        NULLIF(CAST(HCSOC_INV_ACTUAL AS INTEGER), 0),
        NULLIF(CAST(CIEN_INV_ACTUAL AS INTEGER), 0),
        NULLIF(MODULO_INV_ACTUAL, ''),
        
        -- PAES Regular Anterior
        NULLIF(CAST(CLEC_REG_ANTERIOR AS INTEGER), 0),
        NULLIF(CAST(MATE1_REG_ANTERIOR AS INTEGER), 0),
        NULLIF(CAST(MATE2_REG_ANTERIOR AS INTEGER), 0),
        NULLIF(CAST(HCSOC_REG_ANTERIOR AS INTEGER), 0),
        NULLIF(CAST(CIEN_REG_ANTERIOR AS INTEGER), 0),
        NULLIF(MODULO_REG_ANTERIOR, ''),
        
        -- PAES Invierno Anterior
        NULLIF(CAST(CLEC_INV_ANTERIOR AS INTEGER), 0),
        NULLIF(CAST(MATE1_INV_ANTERIOR AS INTEGER), 0),
        NULLIF(CAST(MATE2_INV_ANTERIOR AS INTEGER), 0),
        NULLIF(CAST(HCSOC_INV_ANTERIOR AS INTEGER), 0),
        NULLIF(CAST(CIEN_INV_ANTERIOR AS INTEGER), 0),
        NULLIF(MODULO_INV_ANTERIOR, ''),
        
        -- Indicadores
        RINDIO_PROCESO_ANTERIOR = 1 as rindio_anterior,
        RINDIO_PROCESO_ACTUAL as rindio_actual
        
    FROM read_csv('{DATA_DIR}/ArchivoC_Adm2026REG.csv', 
                  delim=';', 
                  header=true,
                  decimal_separator=',')
""")

# Agregar comentario a la tabla
con.execute("COMMENT ON TABLE resultados_paes IS 'Resultados PAES Admisión 2026 - Puntajes de pruebas y datos académicos de postulantes'")

total = con.execute("SELECT COUNT(*) FROM resultados_paes").fetchone()[0]
print(f"Registros cargados: {total:,}")

## 6. Verificación de Datos

In [ ]:
# Resumen de tablas
print("=== TABLAS EN LA BASE DE DATOS ===")
con.execute("SHOW TABLES").df()

In [ ]:
# Estructura de la tabla principal
print("=== ESTRUCTURA: resultados_paes ===")
con.execute("DESCRIBE resultados_paes").df()

In [ ]:
# Muestra de datos
print("=== MUESTRA DE DATOS ===")
con.execute("""
    SELECT id, rbd, rama, dependencia, puntaje_nem, puntaje_ranking,
           lectora_reg, mate1_reg, mate2_reg, historia_reg, ciencias_reg
    FROM resultados_paes 
    WHERE lectora_reg IS NOT NULL
    LIMIT 10
""").df()

In [ ]:
# Estadísticas básicas
print("=== ESTADÍSTICAS PUNTAJES PAES REGULAR ===")
con.execute("""
    SELECT 
        'Lectora' as prueba,
        COUNT(lectora_reg) as n,
        ROUND(AVG(lectora_reg), 1) as promedio,
        MIN(lectora_reg) as min,
        MAX(lectora_reg) as max
    FROM resultados_paes WHERE lectora_reg IS NOT NULL
    UNION ALL
    SELECT 'Matemática 1', COUNT(mate1_reg), ROUND(AVG(mate1_reg), 1), MIN(mate1_reg), MAX(mate1_reg)
    FROM resultados_paes WHERE mate1_reg IS NOT NULL
    UNION ALL
    SELECT 'Matemática 2', COUNT(mate2_reg), ROUND(AVG(mate2_reg), 1), MIN(mate2_reg), MAX(mate2_reg)
    FROM resultados_paes WHERE mate2_reg IS NOT NULL
    UNION ALL
    SELECT 'Historia', COUNT(historia_reg), ROUND(AVG(historia_reg), 1), MIN(historia_reg), MAX(historia_reg)
    FROM resultados_paes WHERE historia_reg IS NOT NULL
    UNION ALL
    SELECT 'Ciencias', COUNT(ciencias_reg), ROUND(AVG(ciencias_reg), 1), MIN(ciencias_reg), MAX(ciencias_reg)
    FROM resultados_paes WHERE ciencias_reg IS NOT NULL
""").df()

In [ ]:
# Distribución por dependencia
print("=== DISTRIBUCIÓN POR DEPENDENCIA ===")
con.execute("""
    SELECT 
        d.descripcion as dependencia,
        COUNT(*) as postulantes,
        ROUND(AVG(r.puntaje_nem), 1) as prom_nem,
        ROUND(AVG(r.lectora_reg), 1) as prom_lectora
    FROM resultados_paes r
    JOIN ref_dependencia d ON r.dependencia = d.codigo
    GROUP BY d.descripcion
    ORDER BY postulantes DESC
""").df()

In [ ]:
# Cerrar conexión
con.close()
print("Base de datos guardada en: paes.duckdb")